In [4]:
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import degree
from collections import Counter
import matplotlib.pyplot as plt

## Cora Dataset
Cora is a scientific paper citation network of 2708 publications, where each node is a paper and each connection is a reference. Each publication is descried as a binary vector of 1433 unique words (bag of words). The goal is to clssify the category the paper belongs to.

<img src="CoraBalloons.png" atl="cora">
<!-- <img src="cora.png" atl="cora2"> -->

In [16]:
# loading cora dataset
dataset = Planetoid(root=".", name="Cora")
data = dataset[0]

print(f'Dataset: {dataset}')
print(f'-------------------')
print(f'Number of graphs: {len(dataset)}')
# print(f'Number of nodes: {data}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Graph:')
print(f'-------------------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')


Dataset: Cora()
-------------------
Number of graphs: 1
Number of nodes: 2708
Number of features: 1433
Number of classes: 7
Graph:
-------------------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False
